In [1]:
%matplotlib inline
import os
import sys
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset
import torch.optim as optim
from torchmetrics import Accuracy
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter


import matplotlib.pyplot as plt
import numpy as np

# Settings 

torch.set_printoptions(precision=3)

In [2]:
transform_normalize = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                download=True, transform=transform_normalize)

mnist_testset = torchvision.datasets.MNIST(root='./data', train=False,
                                 download=True, transform=transform_normalize)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

train_dataset, val_dataset = torch.utils.data.random_split(mnist_trainset, [50000, 10000])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=64,
                                            shuffle=False, num_workers=2)

valloader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                            shuffle=False, num_workers=2)

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
accuracy = Accuracy(task='multiclass', num_classes=10)

accuracy = accuracy.to(device)
model_alex = AlexNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer_adam = optim.Adam(model_alex.parameters(), lr=0.001)

optimizer_sgd = torch.optim.SGD(model_alex.parameters(), lr=0.001, momentum=0.9)


In [10]:
summary(model=model_alex, input_size=(1, 1, 224, 224), col_width=20,
                  col_names=['input_size', 'output_size', 'num_params', 'trainable'], row_settings=['var_names'], verbose=0)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
AlexNet (AlexNet)                        [1, 1, 224, 224]     [1, 10]              --                   True
├─Sequential (features)                  [1, 1, 224, 224]     [1, 256, 6, 6]       --                   True
│    └─Conv2d (0)                        [1, 1, 224, 224]     [1, 64, 55, 55]      7,808                True
│    └─ReLU (1)                          [1, 64, 55, 55]      [1, 64, 55, 55]      --                   --
│    └─MaxPool2d (2)                     [1, 64, 55, 55]      [1, 64, 27, 27]      --                   --
│    └─Conv2d (3)                        [1, 64, 27, 27]      [1, 192, 27, 27]     307,392              True
│    └─ReLU (4)                          [1, 192, 27, 27]     [1, 192, 27, 27]     --                   --
│    └─MaxPool2d (5)                     [1, 192, 27, 27]     [1, 192, 13, 13]     --                   --
│    └─Conv2d (6)     

In [5]:
# Track the loss and accuracy
timestamp = datetime.now().strftime("%Y-%m-%d")
experiment_name = f'AlexNet_MNIST_{timestamp}'
model_name = 'AlexNet'
log_dir = os.path.join('runs', timestamp, experiment_name, model_name)
log_writer = SummaryWriter(log_dir=log_dir)

In [7]:

for epoch in range(5):  # loop over the dataset multiple times
    train_loss, train_acc = 0, 0
    for X, y in trainloader:
        X, y = X.to(device), y.to(device)

        model_alex.train()
        # model5.train()

        # y_pred = model5(X)
        y_pred = model_alex(X)

        loss = criterion(y_pred, y)
        train_loss += loss.item()

        acc = accuracy(y_pred, y)
        train_acc += acc

        optimizer_adam.zero_grad()
        loss.backward()
        optimizer_adam.step()
    
    train_loss /= len(trainloader)
    train_acc /= len(trainloader)

    val_loss, val_acc = 0, 0
    model_alex.eval()

    with torch.inference_mode():
        for X, y in valloader:
            X, y = X.to(device), y.to(device)
            y_pred = model_alex(X)
            loss = criterion(y_pred, y)
            val_loss += loss.item()
            acc = accuracy(y_pred, y)
            val_acc += acc
        
        val_loss /= len(valloader)
        val_acc /= len(valloader)


    log_writer.add_scalars(main_tag="Loss", tag_scalar_dict={"train/loss": train_loss, "val/loss": val_loss}, global_step=epoch)
    log_writer.add_scalars(main_tag="Accuracy", tag_scalar_dict={"train/acc": train_acc, "val/acc": val_acc}, global_step=epoch)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')


Epoch 1, Loss: 0.0781, Accuracy: 0.9785
Val Loss: 0.0529, Val Accuracy: 0.9857
Epoch 2, Loss: 0.0730, Accuracy: 0.9797
Val Loss: 0.0769, Val Accuracy: 0.9797
Epoch 3, Loss: 0.0634, Accuracy: 0.9821
Val Loss: 0.0906, Val Accuracy: 0.9769
Epoch 4, Loss: 0.0604, Accuracy: 0.9836
Val Loss: 0.0504, Val Accuracy: 0.9848
Epoch 5, Loss: 0.0499, Accuracy: 0.9859
Val Loss: 0.0389, Val Accuracy: 0.9899


In [8]:
torch.save(model_alex.state_dict(), 'alexnet_model_mnist_full.pth')